<a href="https://colab.research.google.com/github/Yahred/evolutionary-computation/blob/main/EvolucionAlgorimoConvLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip freeze > requirements.txt

## Variables

* Número de filtros en las capas ConvLSTM2D:

Para la primera y segunda capa: 16 (puede ser cualquier entero positivo).
Para la tercera capa: también 16 (puede ser cualquier entero positivo).

* Tamaño del kernel en las capas ConvLSTM2D:

Para la primera y segunda capa: (5,5) (puede ser cualquier par de enteros positivos).
Para la tercera capa: (3,3) (puede ser cualquier par de enteros positivos).
* Activación en las capas ConvLSTM2D y Conv2D:

relu para las capas ConvLSTM2D.
sigmoid para la capa Conv2D.
Podrían ser 'tanh', 'softmax', etc.
* Número de canales (channels) en la capa Conv2D:

Depende de tu tarea específica (por ejemplo, 1 para la segmentación binaria, 3 para imágenes RGB).
* Función de pérdida al compilar el modelo:

binary_crossentropy en este ejemplo.
Podrías elegir 'categorical_crossentropy', 'mean_squared_error', etc.
* Optimizador al compilar el modelo:

Adam en este ejemplo.
Podrías elegir 'SGD', 'RMSprop', etc., y también sus hiperparámetros asociados.
* patience para EarlyStopping:

6 en este ejemplo.
Puede ser cualquier entero positivo.
* restore_best_weights para EarlyStopping:

True en este ejemplo.
Puede ser True o False.
* patience para ReduceLROnPlateau:

4 en este ejemplo.
Puede ser cualquier entero positivo.
* Epochs:

20 en este ejemplo.
Puede ser cualquier entero positivo.
* Batch size:

1 en este ejemplo.
Puede ser cualquier entero positivo.

## Librerias y cargar dataset

In [4]:
import os
import cv2
import zipfile
import requests
import zipfile
import io

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from PIL import Image
from google.colab import files
from urllib.request import urlopen
from IPython.display import display, HTML

Se descarga el dataset por medio de una petición HTTP y se extrae


In [23]:
url = 'https://github.com/Yahred/evolutionary-computation/raw/main/data/DroughtDatasetMask.zip'
response = requests.get(url)
data = None

if response.status_code != 200:
  display(HTML((response.text)))
else:
  zip_file = io.BytesIO(response.content)
  data = zipfile.ZipFile(zip_file, 'r')
  data.extractall()

## Funciones

In [24]:
def create_shifted_frames_2(data):
    x = data[:, 0 : data.shape[1] - 1, :, :]
    y = data[:, data.shape[1]-1, :, :]
    return x, y

In [25]:
#Toma todos los colores existentes en la imagen
def get_colors(image):
  aux = []
  band = True
  for i in image:
    for j in i:

      for k in aux:
        if j.tolist() == k:
          band = False
          break
      if band:
        aux.append(j.tolist())
      band = True
  return np.array(aux)

In [26]:
def balance_img_categories(img, palette, balancer):
  #palette = np.sort(palette)
  rows = len(img)
  cols = len(img[0])
  for i in range(rows):
    for j in range(cols):
      pos = np.where(palette == img[i,j])[0][0]
      img[i,j] = balancer[pos]
  return img

In [27]:
#Función para dada una paleta solo tomar los colores de esa paleta en la imagen
def quantizetopalette(silf, palette, dither=False, mode="P"):
  """Convert an RGB or L mode image to use a given P image's palette."""
  silf.load()
  palette.load()
  im = silf.im.convert(mode, 0, palette.im)
  # the 0 above means turn OFF dithering making solid colors
  return silf._new(im)

In [28]:
#Realiza las operaciones necesarias para obtener una imagen RGB por una paleta de colores
def rgb_quantized(img, palette):
  rows, cols = len(img), len(img[0])
  total_vals = 1
  for i in palette.shape:
    total_vals *= i
  palettedata = palette.reshape(total_vals).tolist()
  palImage = Image.new('P', (rows, cols))
  palImage.putpalette(palettedata*32)
  oldImage = Image.fromarray(img).convert("RGB")
  newImage = quantizetopalette(oldImage,palImage)
  res_image = np.asarray(newImage.convert("RGB"))
  return res_image

In [29]:
def gray_quantized(img, palette):
  rows, cols = len(img), len(img[0])
  total_vals = 1
  for i in palette.shape:
    total_vals *= i
  palettedata = palette.reshape(total_vals).tolist()
  palImage = Image.new('L', (rows, cols))
  palImage.putpalette(palettedata*32)
  oldImage = Image.fromarray(img, 'L')
  newImage = quantizetopalette(oldImage,palImage, mode="L")
  res_image = np.asarray(newImage)
  return res_image

In [30]:
def recolor_greys_image(data, palette):
    rows, cols = len(data), len(data[0])
    aux = np.zeros((rows, cols), dtype=np.uint64)
    for i in range(rows):
        for j in range(cols):
            aux[i,j] = min(palette, key= lambda x:abs(x-data[i,j]))
    return aux

In [31]:
def agroup_window(data, window):
    new_data = [data[i:window+i] for i in range(len(data)-window+1)]
    return np.array(new_data)

In [32]:
def add_last(data, new_vals):
    print(f"data: {data.shape} y new_val: {new_vals.shape}")
    x_test_new = data[:,1:]
    print(f"x_test_new: {x_test_new.shape}")

    l = []
    for i in range(len(x_test_new)):
        l.append(np.append(x_test_new[i], new_vals[i]))
    x_test_new = np.array(l).reshape(data.shape[:])
    print("CX", x_test_new.shape)
    return x_test_new

In [33]:
def add_lastNew(data, new_val):
    print(f"data: {data.shape} y new_val: {new_val.shape}")
    x_test_new = data[:,1:,...]  # Omite el primer paso de tiempo
    print(f"x_test_new: {x_test_new.shape}")

    # Asumiendo que new_val es una única predicción que se debe añadir a cada paso de tiempo en x_test_new
    new_val = new_val.squeeze(axis=0)  # Elimina la dimensión del batch, si es necesario

    print(new_val.shape)
    # Añadir new_val a cada elemento en x_test_new
    x_test_new = np.concatenate((x_test_new, np.expand_dims(new_val, axis=1)), axis=1)

    print("CX", x_test_new.shape)
    return x_test_new

In [34]:
#Crea cubos con su propia información de tamaño h
def get_cubes(data, h):
    new_data = []
    for i in range(0, len(data)-h):
        new_data.append(data[i:i+h])
    new_data = np.array(new_data)
    print(new_data.shape)
    return new_data

## Codigo

Variables iniciales para correr el codigo

In [35]:
rows = 122
cols = 360
channels = 1
window = 5
categories = [0, 35, 70, 119, 177, 220, 255]
horizon = 4

De la carpeta con las imagenes pasarlo a un arreglo npy

In [40]:
# Set the path to the folder containing the images
path = "./DroughtDatasetMask"

# Get a list of all the image file names in the folder
image_files = [f for f in os.listdir(path) if f.endswith('.jpg') or f.endswith('.png')]

# Suponiendo que todas las imágenes tienen el tamaño deseado de 122x360
num_images = len(image_files)  # Asegúrate de que este número corresponde al número de imágenes que deseas cargar

# Verifica que tengas la cantidad correcta de archivos de imagen
if len(image_files) != num_images:
    raise ValueError(f"Expected {num_images} images, but found {len(image_files)}")

# Create an empty numpy array to hold the images
images = np.zeros((num_images, rows, cols), dtype=np.uint8)  # Asegúrate de que el tipo de datos sea correcto

# Loop through the image files and add each image to the numpy array
for i, file in enumerate(sorted(image_files)[:num_images]):  # Asegúrate de que no excedas el número de imágenes deseado
    # Load the image using OpenCV
    img = cv2.imread(os.path.join(path, file), cv2.IMREAD_GRAYSCALE)  # Directamente en escala de grises
    if img.shape != (rows, cols):
        raise ValueError(f"The image {file} has a shape of {img.shape}, but expected {(rows, cols)}")
    # Add the image to the numpy array
    images[i] = img

# Save the numpy array to a file
numpy_array_path = './DroughtDatasetMask.npy'
np.save(numpy_array_path, images)

"Images shape: {}".format(images.shape)

'Images shape: (1240, 122, 360)'

cargar dataset en formato npy

In [41]:
x = np.load(numpy_array_path)
print (x.shape)

(1240, 122, 360)


In [42]:
x = np.array([gray_quantized(i, np.array(categories)) for i in x])
colors_greys = get_colors(x[1168])
print(f"Colores {colors_greys}")
print(x.shape)

Colores [  0 255 223 190 215 172 176 212 226 224 169 194  78  69  83 168 181 222
 200 217 177 148 165 213 210 184 218 219  76 139 204 113 198 175 156 133
 205 225 216  90  89 162 193 211 117  82 159 154  70  99 174 187 209 214
 110  98 126 170 220 189 173 182 178 197  72  71 221 104 206 208 138 203
 143 188 161 119 127 150 192 166  68  81  62  52  38  36  48  67  66  34
  59  64  57  50  40 180  77 101 201 167 130  60 129 191 195 179  41 202
 199  94 111]
(1240, 122, 360)


In [ ]:
x_greys = np.array([recolor_greys_image(img, categories) for img in x])
x = x_greys.astype('float32') / 255
print(get_colors(x[1168]))
print(x.shape)

In [ ]:
#Mostrar imágenes
fig, axes = plt.subplots(2, 3, figsize= (10,8))

data_choise = np.random.choice(range(len(x)), size= 1)[0]
for idx, ax in enumerate(axes.flat):
    ax.imshow(np.squeeze(x[data_choise+idx]), cmap='gray')
    ax.set_title(f"Frame {idx + 1}")
    ax.axis("off")
print("Displaying frames for example {}".format(data_choise))
plt.show()

In [ ]:
x_2 = agroup_window(x, window)
print(x_2.shape)
x_train = x_2[:int(len(x_2)*.7)]
x_test = x_2[int(len(x_2)*.7):]
x_validation = x_train[int(len(x_train)*.8):]
x_train = x_train[:int(len(x_train)*.8)]

In [ ]:
x_train = x_train.reshape(len(x_train), window, rows, cols, channels)
x_validation = x_validation.reshape(len(x_validation), window, rows, cols, channels)
x_test = x_test.reshape(len(x_test), window, rows, cols, channels)

print("Forma de datos de entrenamiento: {}".format(x_train.shape))
print("Forma de datos de validación: {}".format(x_validation.shape))
print("Forma de datos de pruebas: {}".format(x_test.shape))

In [ ]:
x_train, y_train = create_shifted_frames_2(x_train)
x_validation, y_validation = create_shifted_frames_2(x_validation)
x_test, y_test = create_shifted_frames_2(x_test)

print("Training dataset shapes: {}, {}".format(x_train.shape, y_train.shape))
print("Validation dataset shapes: {}, {}".format(x_validation.shape, y_validation.shape))
print("Test dataset shapes: {}, {}".format(x_test.shape, y_test.shape))

In [ ]:
np.save("/content/x_test_mask.npy", x_test)
np.save("/content/y_test_mask.npy", y_test)

In [ ]:
#Construction of Convolutional LSTM network
inp = keras.layers.Input(shape=(None, *x_train.shape[2:]))
#It will be constructed a 3 ConvLSTM2D layers with batch normalization,
#Followed by a Conv3D layer for the spatiotemporal outputs.
m = keras.layers.ConvLSTM2D(16, (5,5), padding= "same", return_sequences= True, activation= "relu")(inp)
m = keras.layers.BatchNormalization()(m)
m = keras.layers.ConvLSTM2D(16, (5,5), padding= "same", return_sequences= True, activation= "relu")(m)
m = keras.layers.BatchNormalization()(m)
m = keras.layers.ConvLSTM2D(16, (3,3), padding= "same", activation= "relu")(m)
m = keras.layers.Conv2D(channels, (3,3), activation= "sigmoid", padding= "same")(m)
model = keras.models.Model(inp, m)
model.compile(loss= "binary_crossentropy", optimizer= "Adam")
print(model.summary())
#Callbacks
early_stopping = keras.callbacks.EarlyStopping(monitor= "val_loss", patience= 6, restore_best_weights= True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor= "val_loss", patience= 4)
#Define moifiable training hyperparameters
epochs = 20
batch_size = 2
#Model training
model.fit(
    x_train, y_train,
    batch_size= batch_size,
    epochs= epochs,
    validation_data= (x_validation, y_validation),
    callbacks= [early_stopping, reduce_lr]
)

In [ ]:
#Guardar el modelo
model.save("/content/ConvLSTM2D_Mask122_360.h5")

In [ ]:
imagenInicial = np.random.choice(range(len(x_test)), size= 1)[0]
print(imagenInicial)

In [ ]:
example = x_test[imagenInicial]

print(example.shape)

err = model.evaluate(x_test, y_test, batch_size= 2)
print("El error del modelo es: {}".format(err))
preds = model.predict(x_test, batch_size= 2)
print(preds.shape)
x_test_new = add_last(x_test, preds[:])
preds2 = model.predict(x_test_new, batch_size= 2)
#print(preds2.shape)
x_test_new = add_last(x_test_new, preds2[:])
preds3 = model.predict(x_test_new, batch_size= 2)
x_test_new = add_last(x_test_new, preds3[:])
preds4 = model.predict(x_test_new, batch_size= 2)
res_forecast = add_last(x_test_new, preds4[:])
print("PREDSS",res_forecast.shape)

np.save("/content/PredictionsConvolutionLSTM_forecast_122_360_w5.npy", res_forecast)  #Guardar el vector de predicciones

modelos = []
#agregar pred a modelos
modelos.append(preds)
modelos.append(preds2)
modelos.append(preds3)
modelos.append(preds4)


print("Preds" , preds.shape)
print("Preds2" , preds2.shape)
print("Preds3" , preds3.shape)
print("Preds4" , preds4.shape)
print("Res_forecast" , res_forecast.shape)
print("x_test" , x_test.shape)
print("x_test_new" , x_test_new.shape)
print("y_test" , y_test.shape)

# Selecciona la primera imagen y elimina la dimensión de canal singular con squeeze()
plt.imshow(preds[0].squeeze(), cmap='gray')
plt.title("First Predicted Image")
plt.axis('off')
plt.show()



In [ ]:

def get_Positions(data, rows= 122, cols=360):
    elements = []
    for i in data:
        ix = int(i/cols)
        iy = int(np.round(((i/cols)-ix)*cols))
        elements.append((ix,iy))
    #print(index/cols)
    #print((index/cols)-ix)
    #print(((index/cols)-ix)*cols)
    print("Posiciones!!! {} , {}".format(ix, iy))
    return elements


#Crea cubos con su propia información de tamaño h
def get_cubes(data, h):
    new_data = []
    for i in range(0, len(data)-h):
        new_data.append(data[i:i+h])
    new_data = np.array(new_data)
    print(new_data.shape)
    return new_data

In [ ]:

classes = np.array([0, 255, 220, 177, 119, 70, 35]) # 255, 220, 177, 119, 70, 35  0
classes_rgb = np.array([[0,0,0], [35,35,35], [70,70,70], [119,119,119], [177,177,177], [220,220,220], [255,255,255]])
rows = 122
cols = 360
h = 4

data = np.load("/content/PredictionsConvolutionLSTM_forecast_122_360_w5.npy")
x_test = np.load("/content/x_test_mask.npy")
y_test = np.load("/content/y_test_mask.npy")

print(data.shape)
print(x_test.shape)
print(y_test.shape)

y_test = get_cubes(y_test, h)

colors = get_colors(x_test[-10,0])
print("COLORSS", colors)
print("COLORS", colors.shape)

colorss = get_colors(data[-10,0])
print("COLORSS", colorss)

naive = x_test[:-4]
data = data[1:-3]

#y_real = y_test[:, -h:]*255
new_data = data[:, -h:]
n_real = naive[:, -h:]*255

#y_test = y_test[:, -h:]
naive = naive[:, -h:]

print("XX")
print(y_test.shape)
print(new_data.shape)
print(n_real.shape)

print(min(new_data[0,0,60]))
print(max(new_data[0,0,60]))

new_data = new_data * 255
new_data = new_data.astype(np.uint8)

print("HEY", new_data.shape)
print(colorss.shape)
print(min(new_data[0,0,60]))
print(max(new_data[0,0,60]))

new_data = new_data.reshape(new_data.shape[:-1])
print("HoY", new_data.shape)

aux = []
for i in new_data:
    aux2 = []
    for j in i:
        #res = cv2.cvtColor(j, cv2.COLOR_GRAY2RGB)
        #res = recolor_greys_image(j, classes)
        #rgb_quantized(res, classes_rgb)
        #res = cv2.cvtColor(res, cv2.COLOR_RGB2GRAY)
        res = gray_quantized(j, classes)
        res = recolor_greys_image(res, classes)
        aux2.append(res)
    aux.append(np.array(aux2))
new_data = np.array(aux)
print("SHAPEE", new_data.shape)
color_data = get_colors(new_data[-10,0])
print("DCOLORS", color_data)
new_data = new_data.reshape(new_data.shape[0],new_data.shape[1],new_data.shape[2],new_data.shape[3],1)

#y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2]))*255
#naive = naive.reshape((naive.shape[0], naive.shape[1], naive.shape[2])) * 255

plt.imshow(y_test[0,0], cmap="gray")
#plt.show()


plt.imshow(new_data[0,0], cmap="gray")
#plt.show()


plt.imshow(naive[0,0], cmap="gray")
#plt.show()

y_test = y_test * 255
naive = naive * 255

print("YCOLORS", get_colors(y_test[-10,0]))
print("NCOLORS", get_colors(naive[-10,0]))

print("XS")
print(new_data.shape)
print(y_test.shape)
print(naive.shape)

l_clas = len(classes)

fig = plt.figure(figsize=(20,20))
r = 3
c = 4
ac = 1
pos = 100
for i in range(h):
    fig.add_subplot(r, c, ac)
    ac += 1
    plt.imshow(y_test[pos,i], cmap='gray')
    plt.axis('off')
    plt.title('Original_t+{}'.format(i+1))
for i in range(h):
    fig.add_subplot(r, c, ac)
    ac += 1
    plt.imshow(new_data[pos,i], cmap='gray')
    plt.axis('off')
    plt.title('Pronóstico_t+{}'.format(i+1))
for i in range(h):
    fig.add_subplot(r, c, ac)
    ac += 1
    plt.imshow(naive[pos,i], cmap='gray')
    plt.axis('off')
    plt.title('Naive_t+{}'.format(i+1))

#plt.show()

cm_f = np.zeros((l_clas, l_clas), dtype=np.uint64)
cm_n = np.zeros((l_clas, l_clas), dtype=np.uint64)
print(cm_f)

for e in range(y_test.shape[0]):
    for k in range(h):
        for i in range(rows):
            for j in range(cols):
                pos1 = np.where(classes == y_test[e, k, i, j])[0][0]
                pos2 = np.where(classes == new_data[e, k, i, j])[0][0]
                pos3 = np.where(classes == naive[e, k, i, j])[0][0]
                cm_f[pos1, pos2] += 1
                cm_n[pos1, pos3] += 1

print("Matriz de confusión de pronóstico")
print(cm_f)
print("Matriz de confusión de naive")
print(cm_n)

In [ ]:
fig = plt.figure(figsize=(20,20))
r = 3
c = 4
ac = 1
pos = 100
for i in range(h):
    fig.add_subplot(r, c, ac)
    ac += 1
    plt.imshow(y_test[pos,i], cmap='gray')
    plt.axis('off')
    plt.title('Original_t+{}'.format(i+1))
for i in range(h):
    fig.add_subplot(r, c, ac)
    ac += 1
    plt.imshow(new_data[pos,i], cmap='gray')
    plt.axis('off')
    plt.title('Pronóstico_t+{}'.format(i+1))
for i in range(h):
    fig.add_subplot(r, c, ac)
    ac += 1
    plt.imshow(naive[pos,i], cmap='gray')
    plt.axis('off')
    plt.title('Naive_t+{}'.format(i+1))

plt.show()

## not working

In [ ]:
n_images = 5  # Define cuántas imágenes quieres mostrar
fig, axes = plt.subplots(1, n_images, figsize=(20, 4))
for i in range(n_images):
    for j in range(len(modelos)):
        # Asegúrate de que cada imagen es una matriz 2D al usar squeeze()
        axes[i].imshow(modelos[j][i].squeeze(), cmap='gray')
        axes[i].set_title(f"Predicted Image {i+1}")
        axes[i].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
n_images = 5  # Define cuántas imágenes quieres mostrar
fig, axes = plt.subplots(1, n_images, figsize=(20, 4))
for i in range(n_images):
    #Asegúrate de que cada imagen es una matriz 2D al usar squeeze()
    print(f"Shape of predicted: {preds[i].shape}")
    axes[i].imshow(preds[i].squeeze(), cmap='gray')
    print(f"Shape of predicted image: {preds[i].shape}")
    axes[i].set_title(f"Predicted Image {i+1}")
    axes[i].axis('off')
plt.tight_layout()
plt.show()

In [ ]:
imagenInicial = np.random.choice(range(len(x_test)), size= 1)[0]
print(imagenInicial)

In [ ]:

example = x_test[imagenInicial]
#frames = example[:4, ...]
#original_frames = example[4:, ...]
print(example.shape)
#print(frames.shape)
#print(original_frames.shape)
for _ in range(horizon):
    print(example.shape)
    new_prediction = model.predict(example.reshape(1,*example.shape[0:]))
    example = np.concatenate((example[1:], new_prediction), axis=0)
    print(f"example {example.shape}")
predictions = example[:-3]
print(predictions.shape)
# Selecciona la primera imagen y elimina la dimensión de canal singular con squeeze()
plt.imshow(predictions[0].squeeze(), cmap='gray')
plt.title("First Predicted Image")
plt.axis('off')
plt.show()

In [ ]:

example = x_test[imagenInicial]
#frames = example[:4, ...]
#original_frames = example[4:, ...]
print(example.shape)
#print(frames.shape)
#print(original_frames.shape)
for _ in range(horizon):
    print(example.shape)
    new_prediction = model.predict(example.reshape(1,*example.shape[0:]))
    example = np.concatenate((example[1:], new_prediction), axis=0)
    print(f"example {example.shape}")
predictions = example[:-3]
print(predictions.shape)
# Selecciona la primera imagen y elimina la dimensión de canal singular con squeeze()

for i in range(horizon):
    plt.imshow(example[i].squeeze(), cmap='gray')
    plt.title(f"{i+1} Predicted Image")
    plt.axis('off')
    plt.show()

In [ ]:
x_test = x_test[imagenInicial]
for i in range(horizon):
    plt.imshow(x_test[i].squeeze(), cmap='gray')
    plt.title(f"{i+1} Predicted Image")
    plt.axis('off')
    plt.show()